# Mineração de Dados
 Extração e tratamento de dados obtidos pela API do MyAnimeList jikan

Importação de Biliotecas


In [ ]:
import numpy as np
import pandas as pd
from pandas import json_normalize
from pathlib import Path 
import requests

Extração de dados da API

In [ ]:
url = "https://api.jikan.moe/v4/anime"
df =pd.DataFrame()

pages_qtd = 100

for page in range(1,pages_qtd+1):
    res = requests.get(url,{"page": page}).json()['data']
    df = pd.concat([df,pd.DataFrame(res)])


In [ ]:
print(f"Daframe possui {df.shape[0]} linhas e {df.shape[1]} colunas")
print(f"Colunas: {df.columns}")

Remoção de colunas não uteis

In [ ]:
df = df.drop(columns=['url','images','trailer','approved','titles','title','title_english', 'title_japanese', 
                      'title_synonyms','type','source','aired','members','synopsis','background','season','broadcast',
                      'broadcast', 'producers', 'licensors','studios','explicit_genres','demographics','status'])

In [ ]:
df.isna().any()

Algumas colunas precisam de tratamento:
<ul>
  <li>Algumas linhas possuem Episodes, Rating, Score, Score_by, Rank ou Year assinalado como NaN</li>
  <li>Genres, Themes, Explicit Themes deveme ser convertidas para codificação One Hot</li>
  <li>Duration possui células com formato inconstante</li>
  <li>Rating possui strings que devem ser convertidas para formatos numéricos</li>
</ul>

### Tratamento de valores NaN

In [ ]:
df.loc[df.isna().any(axis=1)]['popularity'].mean()

Como os animes com valores NaN são em média muito pouco poulares, vamos retirá-los do dataframe

In [ ]:
df = df.dropna()
df.shape

### Tratamento de coluna Duration


In [ ]:
df['duration'].unique()

Basta excluir "min per ep" de cada célula

In [ ]:
df['duration'] = pd.to_numeric(df['duration'].str.extract('(\d+)',expand=False))
df

Visualização de valores de "duration"

In [ ]:
df['duration'].unique()

### Rating


In [ ]:
df['rating'].unique()

Conversão de classificações indicativas para valores numéricos

In [ ]:
replacement = {'R - 17+ (violence & profanity)': 18,  
                'R+ - Mild Nudity': 16,
                'PG-13 - Teens 13 or older': 14,
                'PG - Children':10,
                'G - All Ages': 0,
                None: 0
                }
df['rating']=df['rating'].replace(replacement)
df['rating'].unique()

Pelo amor de Deus, vamos retirar hentais do dataset

In [ ]:
drop_rows = df[ df['rating']=='Rx - Hentai' ].index
df.drop(drop_rows , inplace=True)

### One Hot Ecoding

Conversão de json's das células para listas

In [ ]:
df['genres'] =  df['genres'].apply(lambda x: [entry['name'] for entry in x])
df['themes'] =  df['themes'].apply(lambda x: [entry['name'] for entry in x])

Conversão de listas das células para strings

In [ ]:
df['genres'] =  df['genres'].apply(lambda x: ','.join(map(str, x)))
df['themes'] =  df['themes'].apply(lambda x: ','.join(map(str, x)))


Visualização do dataset

In [ ]:
df

One Hot Encoding com gêneros

In [ ]:
one_hot_encoded = df['genres'].str.get_dummies(',')
df = pd.concat([df, one_hot_encoded], axis=1)
df.drop('genres',axis=1)


One Hot Encoding com temas

In [ ]:
one_hot_encoded = df['themes'].str.get_dummies(',')
df = pd.concat([df, one_hot_encoded], axis=1)
df.drop('themes',axis=1)
one_hot_encoded.columns


## Exportação de dataframe

In [ ]:
filepath = Path('./result')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_csv(filepath)